In [1]:
import pandas as pd
from mendeleev.fetch import fetch_table
from source.interactive_plots import interactive_histogram, interactive_violin


In [2]:
xrf_data = pd.read_excel("../data/interim/xrf_data_clean.xlsx")
xrf_data["date_str"] = xrf_data["date"].dt.strftime("%Y-%m-%d")

In [3]:
# get elements analyzed
columns = xrf_data.columns.tolist()
ptable = fetch_table('elements').symbol.to_list()
elements = list(set(columns) & set(ptable))
elements.sort()

In [4]:
dropdown_buttons = {
    "categories": 
        [
            {
                "name": "Session", 
                "column": "date_str", 
                "button row": 1
            },
            {
                "name": "Group", 
                "column": "group", 
                "button row": 1
            }, 
            {
                "name": "Sample Type", 
                "column": "sample_type",
                "button row": 2
            },
            {
                "name": "QA/QC Type", 
                "column": "qaqc_type",
                "button row": 2
            }
        ], 
    "data": 
        {
            "name": "Elements", 
            "columns": elements
        }
    }

In [5]:
interactive_histogram(dropdown_buttons, xrf_data, x_axis_label='Concentration (ppm)', y_axis_label='No. of samples')


In [6]:
interactive_violin(dropdown_buttons, xrf_data, y_axis_label='Concentration (ppm)')
